In [ ]:
import json
import regex
import random

from pathlib import Path
from llama_cpp import Llama
from statistics import mean
from datasets import load_dataset
from huggingface_hub import hf_hub_download

from Libraries import Common_Utils as UTL

In [ ]:
HISTORIES = "Data/Histories.json"
TOKENS_PATH = Path("Config/keys.json")
CONFIG_PATH = Path("Config/config.json")

LLM_CLIENT = None
HF_DATASET = None
REASON_PROMPT = ""
CRITIC_PROMPT = ""
CONFIG = {}

TOKENS = UTL.read_json(TOKENS_PATH)
CONFIG = UTL.read_json(CONFIG_PATH)

HF_REPO_ID = CONFIG["models"]["local_model"]["hf_repo_id"]
HF_FILENAME = CONFIG["models"]["local_model"]["hf_filename"]
CRITICAL_MODEL = CONFIG["models"]["global_model"]["model_name"]

PROMPT_REASON_PATH = Path(CONFIG["paths"]["prompt_reason"])
PROMPT_CRITIC_PATH = Path(CONFIG["paths"]["prompt_critic"])
LOCAL_MODEL_DIR = Path(CONFIG["paths"]["local_model_dir"])
LOCAL_MODEL_PATH = LOCAL_MODEL_DIR / HF_FILENAME

REASON_PROMPT = UTL.read_text(PROMPT_REASON_PATH)
CRITIC_PROMPT = UTL.read_text(PROMPT_CRITIC_PATH)

match (CONFIG, REASON_PROMPT, CRITIC_PROMPT):
    case (False, _, _):
        print("⛔ Dừng chương trình do không tải được file config.")
        exit()
    case (_, False, _) | (_, _, False):
        print("⛔ Dừng chương trình do không tải được file prompt.")
        exit()
    case _:
        print("✅ Tất cả file đã tải thành công.")
        


### HELPERS

In [ ]:
def initialize_dataset():
    """Tải dataset từ Hugging Face."""
    global HF_DATASET
    DATASET_NAME = CONFIG["dataset"]["name"]
    DATASET_SPLIT = CONFIG["dataset"].get("split", "train")
    
    print(f"📘 Đang tải dataset '{DATASET_NAME}' (split: {DATASET_SPLIT})...")
    try:
        HF_DATASET = load_dataset(DATASET_NAME, split=DATASET_SPLIT)
        print(f"✅ Tải dataset thành công. (Số lượng: {len(HF_DATASET)} mẫu)")
    except Exception as e:
        print(f"❌ Lỗi khi tải dataset: {e}")

In [ ]:
def jsonParse(text: str):
    text = text.strip()
    try:
        return json.loads(text)
    except Exception:
        match = regex.search(r"\{(?:[^{}]|(?R))*\}", text, flags=regex.DOTALL)
        if match:
            json_str = match.group(0)
            try:
                return json.loads(json_str)
            except json.JSONDecodeError:
                pass
        raise ValueError(f"❌ Không parse được JSON từ phản hồi:\n{text}")
    
def randomContent() -> str:
    if HF_DATASET is None:
        print("❌ Lỗi: Dataset chưa được tải. (HF_DATASET is None)")
        return None
    try:
        idx = random.randint(0, len(HF_DATASET) - 1)
        sample = HF_DATASET[idx]
        content = sample.get("article") or sample.get("text")
        title = sample.get("headlines", "Không có tiêu đề")
        if not content:
            print(f"⚠️ Mẫu ngẫu nhiên (index {idx}) không có cột 'article' hoặc 'text', thử lại...")
            return randomContent()
        print(f"📘 Chọn ngẫu nhiên mục: {title}")
        return content
    except Exception as e:
        print(f"❌ Lỗi khi lấy mẫu ngẫu nhiên từ dataset: {e}")
        return None

def averageScore(critical_output: dict) -> float:
    """Tính điểm trung bình từ output JSON lồng nhau của critical model."""
    scoring_dict = critical_output.get("scoring")
    if not scoring_dict or not isinstance(scoring_dict, dict):
        if "error" not in critical_output:
             print("⚠️ Lỗi: Không tìm thấy key 'scoring' trong output của Critical.")
        return 0.0
    keys = ["factuality", "clarity", "logical_coherence", "coverage", "utility", "consistency"]
    vals = [scoring_dict[k] for k in keys if isinstance(scoring_dict.get(k), (int, float))]
    return mean(vals) if vals else 0.0

In [ ]:
def initialize_llm_client():
    """Khởi tạo mô hình LLM local duy nhất."""
    global LLM_CLIENT
    print(f"📘 Kiểm tra model local tại: {LOCAL_MODEL_PATH}")
    if not LOCAL_MODEL_PATH.exists():
        print(f"⚠️ Model chưa tồn tại. Bắt đầu tải '{HF_FILENAME}' từ '{HF_REPO_ID}'...")
        LOCAL_MODEL_DIR.mkdir(parents=True, exist_ok=True)
        try:
            hf_hub_download(
                repo_id=HF_REPO_ID,
                filename=HF_FILENAME,
                local_dir=LOCAL_MODEL_DIR,
                local_dir_use_symlinks=False,
                resume_download=True
            )
            print("✅ Tải model thành công.")
        except Exception as e:
            print(f"❌ Lỗi khi tải model: {e}")
            return 
    else:
        print("👍 Model đã có sẵn.")

    print(f"📘 Đang tải model local từ: {LOCAL_MODEL_PATH}")
    try:
        LLM_CLIENT = Llama(
            model_path=str(LOCAL_MODEL_PATH),
            n_gpu_layers=CONFIG["llama_cpp_params"]["n_gpu_layers"],
            n_ctx=CONFIG["llama_cpp_params"]["n_ctx"],
            verbose=False
        )
        print("✅ Tải model local (LLM_CLIENT) thành công.")
    except Exception as e:
        print(f"❌ Lỗi khi tải model local: {e}")

### RUNNER

In [ ]:
def reasoningRun(text: str, feedback: str | None = None) -> str:
    """Chạy reasoning. Nếu có feedback, nó sẽ chạy ở chế độ 'refine'."""
    if LLM_CLIENT is None:
        print("❌ Lỗi: reasoningRun không thể chạy vì LLM_CLIENT chưa được tải.")
        return ""
        
    if feedback:
        prompt = (
            f"<|user|>\nBạn đã tạo ra một bản tóm tắt trước đó. "
            f"Bản tóm tắt đó đã được đánh giá với phản hồi sau:\n"
            f"---BEGIN FEEDBACK---\n{feedback}\n---END FEEDBACK---\n\n"
            f"Vui lòng tạo lại reasoning trace và tóm tắt, có tính đến phản hồi này. "
            f"Sử dụng lại văn bản gốc dưới đây.\n\n"
            f"Văn bản gốc:\n{text}<|end|>\n<|assistant|>"
        )
    else:
        prompt = f"<|user|>\n{REASON_PROMPT}\n\n{text}<|end|>\n<|assistant|>"
    
    print(f"prompt (local): {prompt}...")

    try:
        completion = LLM_CLIENT(
            prompt,
            max_tokens=CONFIG["generation_params"]["max_new_tokens"],
            temperature=CONFIG["generation_params"]["temperature"],
            top_p=CONFIG["generation_params"]["top_p"],
            stop=["<|end|>", "<|endoftext|>", "</s>"] 
        )
        
        if "choices" in completion and len(completion["choices"]) > 0:
            return completion["choices"][0]["text"].strip()
        else:
            return str(completion).strip()

    except Exception as e:
        print(f"❌ Lỗi khi chạy reasoningRun (local): {e}")
        return ""

In [ ]:
def criticalRun(source_text: str, reasoning_output: str) -> dict:
    """Chạy critical BẰNG MÔ HÌNH LOCAL, trả về dict JSON."""
    if LLM_CLIENT is None:
        return {"error": "LLM client not initialized"}
    
    user_prompt = f"""
    {CRITIC_PROMPT}

    [Reasoning trace]:
    {reasoning_output}

    [Văn bản gốc]:
    {source_text}
    """
    
    final_prompt = f"<|user|>\n{user_prompt}<|end|>\n<|assistant|>"

    try:
        completion = LLM_CLIENT(
            final_prompt,
            max_tokens=1024,
            temperature=0.2,
            top_p=0.9,
            stop=["<|end|>", "```"]
        )

        if "choices" not in completion or len(completion["choices"]) == 0:
            raise ValueError("Mô hình local (critical) không trả về nội dung.")

        raw_output = completion["choices"][0]["text"].strip()
        if not raw_output:
            raise ValueError("Mô hình local (critical) trả về phản hồi rỗng.")

        try:
            parsed_json = jsonParse(raw_output)
            return parsed_json
            
        except ValueError as e:
            print(f"⚠️ Lỗi parse JSON từ mô hình local: {e}")
            print(f"--- RAW OUTPUT TỪ PHI-3 (CRITICAL) ---")
            print(raw_output)
            print(f"----------------------------------------")
            return {"error": "Failed to parse JSON from local model", "raw_response": raw_output}

    except Exception as e:
        error_message = f"❌ Lỗi khi gọi LLM (critical): {str(e)}"
        print(error_message)
        return {"error": error_message}

### MAIN FUNC

In [ ]:
def mainFlow(source_text: str, 
             max_iters=CONFIG["flow_params"]["max_iters"], 
             min_improve=CONFIG["flow_params"]["min_improve"]):
    
    best_reason, best_score = None, 0
    history = []
    current_feedback = None
    current_reasoning = ""

    for step in range(1, max_iters + 1):
        print(f"\n🔄 Vòng {step} ...")
        
        current_reasoning = reasoningRun(source_text, current_feedback)
        
        artifact_marker = "CV=" 
        if artifact_marker in current_reasoning:
            print(f"⚠️ Phát hiện artifact '{artifact_marker}', đang cắt bỏ...")
            artifact_index = current_reasoning.find(artifact_marker)
            current_reasoning = current_reasoning[:artifact_index].strip()
        
        if not current_reasoning:
            print("⛔ Reasoning trả về rỗng, dừng vòng lặp.")
            break
            
        print(f"\n🔄 Reaoning Result: {current_reasoning}")
        
        critical_output = criticalRun(source_text, current_reasoning)
        
        if critical_output.get("error"):
            print(f"⛔ Lỗi từ Critical: {critical_output['error']}, dừng vòng lặp.")
            break
        
        average_score = averageScore(critical_output)
        
        current_feedback = critical_output.get("feedback_text", "") 

        print(f"📊 Điểm TBC: {average_score:.2f}")
        print(f"📝 Nhận xét (Toàn bộ JSON): {json.dumps(critical_output, ensure_ascii=False, indent=2)}\n")

        history.append({
            "round": step, 
            "score": average_score, 
            "reasoning": current_reasoning, 
            "evaluation": critical_output.get("scoring", {}),
            "feedback": current_feedback
        })
        
        if average_score > best_score + min_improve:
            best_reason, best_score = current_reasoning, average_score
            print(f"📈 Cải thiện tốt, điểm mới: {best_score:.2f}")
            if average_score >= 4.8:
                print("🎉 Đạt điểm cao, dừng sớm.")
                break
        else:
            print("⛔ Không cải thiện đáng kể, dừng.")
            break

    return {"best_reasoning": best_reason, "best_score": best_score, "history": history}

### RUN MAIN

In [ ]:
if __name__ == "__main__":
    
    # --- GỌI CÁC HÀM KHỞI TẠO ---
    initialize_llm_client()
    initialize_dataset()
    
    # ⬅️ Chỉ cần kiểm tra 2 thành phần
    if LLM_CLIENT is None or HF_DATASET is None: 
        print("⛔ Không thể bắt đầu mainFlow (model hoặc dataset chưa được tải).")
        print("Vui lòng kiểm tra lại lỗi ở trên.")
        exit()
        
    try:
        inputPara = randomContent()
        if inputPara is None:
            print("⛔ Không tải được nội dung test data, dừng chương trình.")
            exit()
            
        flow_params = CONFIG.get("flow_params", {})
        result = mainFlow(
            inputPara,
            max_iters=flow_params.get("max_iters", 3),
            min_improve=flow_params.get("min_improve", 0.05)
        )

        print("\n" + "="*50)
        print("🎯 KẾT QUẢ CUỐI CÙNG 🎯")
        print("="*50)
        
        final_output = {
            "best_score": result["best_score"],
            "best_reasoning": result["best_reasoning"]
        }
        
        print(json.dumps(final_output, ensure_ascii=False, indent=2))
        
        UTL.insert_json(result["history"], HISTORIES, indent=2)
        print(f"\n📝 Lịch sử chạy đầy đủ đã được lưu vào '{HISTORIES}'")

    except Exception as e:
        print(f"\n❌ Đã xảy ra lỗi không mong muốn trong main: {e}")